# Ivy Database

We have a SQL-server based databased named as Ivy Database in the FRE department. Yet the access may be closed on holidays and is only limited to NYU network even if it is available. Here I provide a demo for retrieve data from this database.

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pymssql
import pandas as pd
import numpy as np

conn = pymssql.connect(server="fresql.poly.edu", user="ivyuser", password="resuyvi")
sql1 = '''
SELECT sp.Date date, 
sp.SecurityID,
s.Ticker ticker, 
sp.ClosePrice price
FROM XFDATA.DBO.SECURITY_PRICE AS SP
JOIN XFDATA.DBO.SECURITY AS S ON sp.SecurityID = S.SecurityID
WHERE SP.Date BETWEEN '2021-03-01' AND '2021-03-01'
order by 
s.Ticker,  
sp.Date
'''
data = pd.read_sql(sql1, conn)
data

,date,SecurityID,ticker,price
0,2021-03-01,109811,,24.540001
1,2021-03-01,102479,,0.000000
2,2021-03-01,209477,,114.989998
3,2021-03-01,101149,A,124.480003
4,2021-03-01,210354,AA,25.990000
...,...,...,...,...
8721,2021-03-01,128292,ZZZZ,0.000000
8722,2021-03-01,134880,ZZZZ,80.853302
8723,2021-03-01,128350,ZZZZ,0.010000
8724,2021-03-01,129857,ZZZZ,604.389221


Below I follow instructions from [Stefan Jansen's github](https://github.com/stefan-jansen/machine-learning-for-trading) to query data from open resources.

# Select tickers

I first get the list for S&P 500 stocks. I use historical data for these stocks to create my machine learning dataset.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]
df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
ticker_list = df['Symbol'].tolist()
ticker_str = " ".join(ticker_list)
ticker_str

'MMM AOS ABT ABBV ABMD ACN ATVI ADM ADBE ADP AAP AES AFL A APD AKAM ALK ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AMD AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ANET AJG AIZ T ATO ADSK AZO AVB AVY BKR BALL BAC BBWI BAX BDX WRB BRK.B BBY BIO TECH BIIB BLK BK BA BKNG BWA BXP BSX BMY AVGO BR BRO BF.B CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE CNC CNP CDAY CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DISH DIS DG DLTR D DPZ DOV DOW DTE DUK DD DXC EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY RE EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FAST FRT FDX FITB FRC FE FIS FISV FLT FMC F FTNT FTV FBHS FOXA FOX BEN FCX GRMN IT GNRC GD GE GIS GM GPC GILD GL GPN GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HU

# yfinance

I use `yfinance` to collect data from yahoo finance for the listed stocks above.

In [5]:
import pandas as pd
import yfinance as yf

tickers = yf.Tickers(ticker_str)


In [6]:
tickers.tickers

{'MMM': yfinance.Ticker object <MMM>,
 'AOS': yfinance.Ticker object <AOS>,
 'ABT': yfinance.Ticker object <ABT>,
 'ABBV': yfinance.Ticker object <ABBV>,
 'ABMD': yfinance.Ticker object <ABMD>,
 'ACN': yfinance.Ticker object <ACN>,
 'ATVI': yfinance.Ticker object <ATVI>,
 'ADM': yfinance.Ticker object <ADM>,
 'ADBE': yfinance.Ticker object <ADBE>,
 'ADP': yfinance.Ticker object <ADP>,
 'AAP': yfinance.Ticker object <AAP>,
 'AES': yfinance.Ticker object <AES>,
 'AFL': yfinance.Ticker object <AFL>,
 'A': yfinance.Ticker object <A>,
 'APD': yfinance.Ticker object <APD>,
 'AKAM': yfinance.Ticker object <AKAM>,
 'ALK': yfinance.Ticker object <ALK>,
 'ALB': yfinance.Ticker object <ALB>,
 'ARE': yfinance.Ticker object <ARE>,
 'ALGN': yfinance.Ticker object <ALGN>,
 'ALLE': yfinance.Ticker object <ALLE>,
 'LNT': yfinance.Ticker object <LNT>,
 'ALL': yfinance.Ticker object <ALL>,
 'GOOGL': yfinance.Ticker object <GOOGL>,
 'GOOG': yfinance.Ticker object <GOOG>,
 'MO': yfinance.Ticker object <MO>

In [7]:
data = pd.DataFrame([])

for ticker in ticker_list:

    t = tickers.tickers[ticker].history(period="max")
    t["Ticker"] = ticker

    data = pd.concat([data, t], axis=0)

data

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close
Date,,,,,,,,,
1962-01-02,0.000000,0.771044,0.748367,0.754036,212800.0,0.0,0.0,MMM,NaN
1962-01-03,0.000000,0.759705,0.741280,0.759705,422400.0,0.0,0.0,MMM,NaN
1962-01-04,0.000000,0.772462,0.759705,0.759705,212800.0,0.0,0.0,MMM,NaN
1962-01-05,0.000000,0.756871,0.737028,0.739863,315200.0,0.0,0.0,MMM,NaN
1962-01-08,0.000000,0.741280,0.731358,0.735610,334400.0,0.0,0.0,MMM,NaN
...,...,...,...,...,...,...,...,...,...
2022-09-30,150.419998,152.029999,148.039993,148.289993,2437200.0,0.0,0.0,ZTS,NaN
2022-10-03,149.279999,152.850006,148.309998,151.350006,1580200.0,0.0,0.0,ZTS,NaN
2022-10-04,153.240005,155.309998,152.600006,154.750000,1676300.0,0.0,0.0,ZTS,NaN


In [8]:
data.to_csv("price.csv")